За каждую заключенную сделку менеджер получает бонус, который рассчитывается следующим образом:
1) За каждую `новую` сделку менеджер получает **7%** от суммы сделки, при условии, что статус оплаты `ОПЛАЧЕНО`, а также имеется оригинал подписанного договора с клиентом;
2) За каждую `текущую` сделку менеджер получает **5%** от суммы сделки, если она больше `10 тыс.`, и **3%** от суммы, если меньше. При этом статус оплаты может быть любым, кроме `ПРОСРОЧЕНО`, а также необходимо наличие оригинала подписанного договора с клиентом

Бонусы по сделкам, оригиналы для которых приходят позже рассматриваемого месяца, считаются остатком на следующий период, который выплачивается по мере прихода оригиналов. 

Вычислите остаток каждого из менеджеров на 01.07.2021.

### Импорт данных

Импортируем исходный набор данных и удалим ненужный столбец

In [30]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_excel("data.xlsx")
del df["Unnamed: 5"]
df

,client_id,sum,status,sale,new/current,document,receiving_date
0,NaN,NaN,Май 2021,NaN,NaN,NaN,NaN
1,6.0,11693.7,ВНУТРЕННИЙ,-,-,-,-
2,14.0,5452.7,ОПЛАЧЕНО,Петрова,новая,оригинал,2021-07-26 00:00:00
3,15.0,13991.7,ОПЛАЧЕНО,Иванов,текущая,оригинал,2021-06-08 00:00:00
4,16.0,719.7,ОПЛАЧЕНО,Иванов,текущая,оригинал,2021-06-08 00:00:00
...,...,...,...,...,...,...,...
725,285.0,4328.7,ОПЛАЧЕНО,Васильев,текущая,оригинал,2021-11-15 00:00:00
726,360.0,3096.2,ОПЛАЧЕНО,Васильев,новая,НЕТ,NaN
727,360.0,3096.2,ПРОСРОЧЕНО,Васильев,новая,НЕТ,NaN
728,373.0,4403.7,ОПЛАЧЕНО,Михайлов,новая,оригинал,2021-11-08 00:00:00


Далее приведём тип столбца с датой к типу данных даты

In [31]:
df = df.dropna(subset=["receiving_date"])
df = df.iloc[1:,:]
df["receiving_date"] = pd.to_datetime(df["receiving_date"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 606 entries, 2 to 729
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   client_id       606 non-null    float64       
 1   sum             606 non-null    float64       
 2   status          606 non-null    object        
 3   sale            606 non-null    object        
 4   new/current     606 non-null    object        
 5   document        595 non-null    object        
 6   receiving_date  606 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 37.9+ KB


Отфильтруем полученные данные таким образом, чтобы дата получения оригинала была в июне

In [32]:
df = df.query("'2021-06-01' <= receiving_date <= '2021-06-30'")
df

,client_id,sum,status,sale,new/current,document,receiving_date
3,15.0,13991.7,ОПЛАЧЕНО,Иванов,текущая,оригинал,2021-06-08
4,16.0,719.7,ОПЛАЧЕНО,Иванов,текущая,оригинал,2021-06-08
8,23.0,8511.7,ОПЛАЧЕНО,Андреев,текущая,оригинал,2021-06-08
9,24.0,16347.7,ОПЛАЧЕНО,Андреев,новая,оригинал,2021-06-23
10,28.0,10537.7,ОПЛАЧЕНО,Андреев,текущая,оригинал,2021-06-16
...,...,...,...,...,...,...,...
120,266.0,3363.7,ОПЛАЧЕНО,Петрова,текущая,оригинал,2021-06-23
121,270.0,5288.7,ОПЛАЧЕНО,Васильев,текущая,оригинал,2021-06-07
122,273.0,10133.7,ОПЛАЧЕНО,Петрова,текущая,оригинал,2021-06-01
123,274.0,26033.7,ОПЛАЧЕНО,Петрова,текущая,оригинал,2021-06-22


### Решение

Остаток на 01.07.2021 - это означает посчитать остаток всех сделок, оригиналы документов которых были принесены до 01.07.2021, т.е. в мае-июне. При этом, если сделка была совершена в мае и в мае же были принесены документы - то такие сделки не идут в счет.

Пример 1:

| client | sum      | status   | sale    | new/current | document | date       |
|--------|----------|----------|---------|-------------|----------|------------|
| 14     | 5,452.70 | ОПЛАЧЕНО | Петрова | новая       | оригинал | 26.07.2021 |

Сделка новая, значит менеджер Петрова получит 7% от 5,452.70, поскольку статус `ОПЛАЧЕНО`. Однако бонус будет начислен лишь на июль. А по условию необходимо найти остаток на начало июня.


Пример 2:

| client | sum       | status   | sale    | new/current | document | date       |
|--------|-----------|----------|---------|-------------|----------|------------|
| 15     | 13,991.70 | ОПЛАЧЕНО | Иванов  | текущая     | оригинал | 08.06.21   |

Сделка текущая и превышает 10 тыс., статус `ОПЛАЧЕНО`, значит менеджер получит 5% от 13,991.70. Бонус будет начислен в июне, значит данный пример подходит под условие задачи.

Пример 3:

| client | sum       | status   | sale    | new/current | document | date       |
|--------|-----------|----------|---------|-------------|----------|------------|
| 94     | 31,094.20 | ОПЛАЧЕНО | Андреев | текущая     | оригинал | 18.05.21   |

Сделка текущая, превышает 10 тыс., статус `ОПЛАЧЕНО` => менеджер получит 5% от 31,094.20 в мае 

Пример 4:

| client | sum       | status   | sale    | new/current | document | date       |
|--------|-----------|----------|---------|-------------|----------|------------|
| 87     | 9,948.70  | ОПЛАЧЕНО | Смирнов | новая       |          | 18.05.21   |

Сделка новая, статус `ОПЛАЧЕНО`, однако отсутствует оригинал => менеджер ничего не получает


Для начала создадим словарь, ключами которого являются фамилии менеджеров, а значениями - размер бонуса на 01.07.2021

In [9]:
managers = df["sale"].unique().tolist()
managers = [elem for elem in managers if elem != '-' and isinstance(elem, str)]

bonus = dict((surname, 0) for surname in managers)

{'Петрова': 0,
 'Иванов': 0,
 'Кузнецова': 0,
 'Андреев': 0,
 'Филимонова': 0,
 'Селиванов': 0,
 'Смирнов': 0,
 'Васильев': 0,
 'Соколов': 0,
 'Михайлов': 0,
 'Попов': 0}

В цикле пройдемся по всем элементам из датафрейма и посчитаем размер бонуса для каждого менеджера

In [36]:
for row in df.itertuples(index=False):
    if row._4 == "новая":
        if row.status == 'ОПЛАЧЕНО' and row.document == "оригинал":
            bonus[row.sale] += 0.07 * row.sum
    else:
        if row.status != 'ПРОСРОЧЕНО' and row.document == 'оригинал':
            if row.sum > 10_000:
                bonus[row.sale] += 0.05 * row.sum
            else:
                bonus[row.sale] += 0.03 * row.sum

И выведем окончательные размеры бонусов для каждого менеджера

In [37]:
bonus

{'Петрова': 7028.8020000000015,
 'Иванов': 2503.839,
 'Кузнецова': 2604.6715999999997,
 'Андреев': 2962.1717,
 'Филимонова': 306.072,
 'Селиванов': 353.322,
 'Смирнов': 4078.724,
 'Васильев': 339.231,
 'Соколов': 0,
 'Михайлов': 0,
 'Попов': 0}